# Phase 2: Modeling and Explainability

This notebook follows the exploratory data analysis (EDA) and focuses on modeling, evaluation, and interpretability. 
The main objectives are:

1. **Cluster-based profiling** of tourists according to trip characteristics and expenditure patterns.  
2. **Prediction of key variables** such as daily average expenditure and trip length.  
3. **Explainability using SHAP values**, to make the models transparent and suitable for reporting in the chatbot-based system.  

The prepared dataset (`egatur_data_ready.csv`) created in the EDA phase will be the starting point. 

Load all required libraries, including machine learning, visualization, and explainability tools. Also, load the processed dataset from the previous EDA phase.

In [ ]:
# ===============================
# 1. Setup & Data Loading
# ===============================
import pandas as pd
import numpy as np
from pathlib import Path

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_validate, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report
from sklearn.decomposition import PCA
import seaborn as sns
from adjustText import adjust_text
import matplotlib.pyplot as plt


# Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.neural_network import MLPClassifier


# Explainability
import shap

# Saving
import joblib

# Paths
DATA_DIR = Path("../data/processed")
df = pd.read_csv(DATA_DIR / "egatur_data_ready.csv")

df.head()

,daily_average_expenditure,trip_length,accommodation,purpose,season,country,region,cluster
0,129.318995,14,Non-market,Leisure/holidays,winter,Occident and America,Valencian Community,0
1,127.224771,16,Non-market,Leisure/holidays,winter,Occident and America,Valencian Community,0
2,273.413983,9,Hotels and similar,Leisure/holidays,winter,Occident and America,Madrid,3
3,111.493501,27,Non-market,Leisure/holidays,winter,Occident and America,Valencian Community,0
4,124.846663,14,Non-market,Other,winter,Occident and America,Canary Islands,2


## 2. Data Preparation

The dataset contains both numerical and categorical variables.  
For modeling purposes, we will:
- Scale numerical variables (`daily_average_expenditure_log`, `trip_length_log`).
- Encode categorical variables (`accommodation`, `purpose`, `season`, `country`, `region`).
- Separate target variables for supervised models.

Define feature columns (inputs) and target columns (outputs) for regression and clustering models.

In [2]:
# Features and targets
features = ["accommodation", "purpose", "season", "country", "region",
            "daily_average_expenditure", "trip_length"]

target_expenditure = "daily_average_expenditure"
target_trip_length = "trip_length"

X = df[features]
y_expenditure = df[target_expenditure]
y_trip_length = df[target_trip_length]

# Preprocessing pipeline
numeric_features = ["daily_average_expenditure", "trip_length"]
categorical_features = ["accommodation", "purpose", "season", "country", "region"]

# Transformer: log(1+x) to avoid log(0)
def log_transform(x):
    return np.log1p(x)
log_transformer = FunctionTransformer(log_transform, validate=False)

numeric_pipeline = Pipeline(steps=[
    ("log", log_transformer),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore",
                             categories=[df[col].unique() for col in categorical_features])),
    ("scaler", StandardScaler(with_mean=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numeric_features),
        ("cat", categorical_pipeline, categorical_features)
    ]
)

We have built two pipelines for numeric and categorical variables to ensure consistent preprocessing:

- Numeric: log transformation + standard scaling  
- Categorical: one-hot encoding + scaling (to normalize magnitudes for algorithms sensitive to feature scale)  

## 3. Clustering with classificator

We want a reproducible model that predicts the same clusters obtained in the EDA.
By training a neural network to replicate the original cluster labels, we can apply
it to new data while keeping the same segmentation logic.

In [ ]:
# Features and target
X_clusters = df[features]
y_clusters = df["cluster"]  # clusters from the original EDA

# Train/test split (stratified to preserve cluster proportions)
X_train, X_test, y_train, y_test = train_test_split(
    X_clusters, y_clusters, test_size=0.2, random_state=42, stratify=y_clusters
)

# Neural network classifier pipeline
nn_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),  # same preprocessing as clustering pipeline
    ("classifier", MLPClassifier(
        hidden_layer_sizes=(100,50),
        activation="relu",
        solver="adam",
        max_iter=500,
        random_state=42
    ))
])

# Train the neural network
nn_pipeline.fit(X_train, y_train)

# Predict on test set
y_pred = nn_pipeline.predict(X_test)

# Evaluate performance
print("Classification report for EDA clusters (Neural Network):")
print(classification_report(y_test, y_pred))

# Apply to full dataset to generate reproducible cluster labels
df["cluster_pred_nn"] = nn_pipeline.predict(X_clusters)

# Save the neural network classifier
joblib.dump(nn_pipeline, "../models/cluster/cluster_classifier_eda_nn.joblib")

# Quick summary of predicted clusters
print("\nCluster counts (supervised NN prediction):")
print(df["cluster_pred_nn"].value_counts())

Classification report for EDA clusters (Neural Network):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6426
           1       1.00      1.00      1.00     20672
           2       1.00      1.00      1.00      6909
           3       1.00      1.00      1.00     13151

    accuracy                           1.00     47158
   macro avg       1.00      1.00      1.00     47158
weighted avg       1.00      1.00      1.00     47158


Cluster counts (supervised NN prediction):
cluster_pred_nn
1    103573
3     65538
2     34560
0     32117
Name: count, dtype: int64


The neural network achieves an almost perfect accuracy when reproducing the EDA clusters.  
This is expected because the `labels` (clusters) come from a deterministic algorithm (KMeans) and are fully separable given the preprocessing.  
Therefore, the network is not discovering new patterns but rather **learning to replicate the segmentation logic** established during the EDA.  
This ensures consistency: when applying the model to new data, the cluster assignment will be aligned with the original exploratory segmentation.

In [4]:
# We define the stratified CV (to preserve the proportion of clusters)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluation with accuracy
cv_scores = cross_val_score(nn_pipeline, X_clusters, y_clusters, cv=cv, scoring="accuracy", n_jobs=4)

print("Cross-validation accuracy scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())


Cross-validation accuracy scores: [0.99817634 0.99830358 0.99830358 0.9988973  0.99832475]
Mean CV accuracy: 0.9984011078588807


To confirm the robustness of the model, we applied a 5-fold stratified cross-validation.  
The results show accuracies consistently above 99.8%, confirming that the clusters are **stable and easy to reproduce** across different data splits.  
This reinforces that the neural network generalizes the cluster structure rather than simply memorizing the training set.

In summary, the neural network classifier serves as a **reproducible mapping** from raw trip features to the EDA-defined clusters.  
This allows us to:  
- Preserve the interpretability of the original segmentation.  
- Ensure new incoming data can be assigned to the same clusters.  
- Integrate this logic into downstream tasks (e.g., supervised modeling, chatbot explanations).  

The near-perfect accuracy should not be interpreted as overfitting, but as a confirmation that the EDA segmentation is fully recoverable given the available features.

## 4. Supervised Regression Models

After exploring several modeling options, we focus on a single supervised regression task: predicting **average daily expenditure** using all available trip features. This approach is most relevant for our application, as it enables us to classify and profile tourists by their spending behavior—an essential insight for downstream reporting and chatbot explanations.

In [8]:
# Define target
target_col = "daily_average_expenditure"

y = df[target_col]
X = df[[f for f in features if f != target_col]]

cat_cols = [col for col in X.columns if col in categorical_features]
num_cols = [col for col in X.columns if col in numeric_features]

# Preprocesador
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
    ("num", StandardScaler(), num_cols)
])

We try out a few different models to predict daily average expenditure:

- Linear Regression as a simple baseline  
- Random Forest for flexible, non-linear relationships  
- Gradient Boosting for stronger ensemble performance  
- HistGradientBoosting for fast and efficient boosting  

We use cross-validation with R2, MAE, and RMSE to get a fair comparison of how each model performs.

In [9]:
# Models to evaluate (baseline configs)
models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(random_state=42, n_estimators=100),
    "HistGradientBoosting": HistGradientBoostingRegressor(random_state=42)
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)
results = []

for model_name, model in models.items():
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("regressor", model)
    ])
    scores = cross_validate(
        pipe, X, y,
        cv=cv,
        scoring=["r2", "neg_mean_absolute_error", "neg_root_mean_squared_error"],
        return_train_score=False,
        n_jobs=4
    )
    results.append({
        "model": model_name,
        "R2_mean": np.mean(scores["test_r2"]),
        "MAE_mean": -np.mean(scores["test_neg_mean_absolute_error"]),
        "RMSE_mean": -np.mean(scores["test_neg_root_mean_squared_error"])
    })

results_df = pd.DataFrame(results)
results_df

,model,R2_mean,MAE_mean,RMSE_mean
0,LinearRegression,0.410654,75.139150,155.240273
1,RandomForest,0.659895,49.298094,117.919709
2,GradientBoosting,0.666203,49.127912,116.829239
3,HistGradientBoosting,0.672925,48.292829,115.644059


As it is not the main problem we want to explore, we pick the best model (HistGradientBoosting) and fine-tune its hyperparameters with `GridSearchCV` to squeeze out extra performance without changing the overall model type.

In [11]:
pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", HistGradientBoostingRegressor(random_state=42))
])

param_grid = {
    "regressor__learning_rate": [0.01, 0.05, 0.1],
    "regressor__max_iter": [100, 200, 500],
    "regressor__max_depth": [None, 5, 10],
    "regressor__min_samples_leaf": [20, 50, 100],
    "regressor__l2_regularization": [0.0, 0.1, 1.0]
}

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=2
)

grid.fit(X, y)

print("Mejores parámetros:", grid.best_params_)
print("Mejor R2 CV:", grid.best_score_)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END regressor__l2_regularization=0.0, regressor__learning_rate=0.01, regressor__max_depth=None, regressor__max_iter=100, regressor__min_samples_leaf=20; total time=  10.5s
[CV] END regressor__l2_regularization=0.0, regressor__learning_rate=0.01, regressor__max_depth=None, regressor__max_iter=100, regressor__min_samples_leaf=20; total time=  12.3s
[CV] END regressor__l2_regularization=0.0, regressor__learning_rate=0.01, regressor__max_depth=None, regressor__max_iter=100, regressor__min_samples_leaf=20; total time=  12.4s
[CV] END regressor__l2_regularization=0.0, regressor__learning_rate=0.01, regressor__max_depth=None, regressor__max_iter=100, regressor__min_samples_leaf=20; total time=  15.7s
[CV] END regressor__l2_regularization=0.0, regressor__learning_rate=0.01, regressor__max_depth=None, regressor__max_iter=100, regressor__min_samples_leaf=20; total time=  12.2s
[CV] END regressor__l2_regularization=0.0, regresso

In [12]:
# Get the best model
best_model = grid.best_estimator_

# Save 
joblib.dump(best_model, "../models/daily_avg_exp/histgradientboosting_model.joblib")

['../models/daily_avg_exp/histgradientboosting_model.joblib']

In [13]:
scores = cross_validate(
    best_model,
    X, y,
    cv=5,
    scoring=["r2", "neg_mean_absolute_error", "neg_root_mean_squared_error"],
    return_train_score=False,
    n_jobs=-1
)

tuned_results = {
    "model": "HistGradientBoosting (tuned)",
    "R2_mean": np.mean(scores["test_r2"]),
    "MAE_mean": -np.mean(scores["test_neg_mean_absolute_error"]),
    "RMSE_mean": -np.mean(scores["test_neg_root_mean_squared_error"])
}

results_df = pd.concat([results_df, pd.DataFrame([tuned_results])], ignore_index=True)

results_df

,model,R2_mean,MAE_mean,RMSE_mean
0,LinearRegression,0.410654,75.139150,155.240273
1,RandomForest,0.659895,49.298094,117.919709
2,GradientBoosting,0.666203,49.127912,116.829239
3,HistGradientBoosting,0.672925,48.292829,115.644059
4,HistGradientBoosting (tuned),0.673479,48.355998,115.521238


Despite hyperparameter tuning, the final model only improved slightly over the baseline HistGradientBoosting, indicating that further gains may require additional features or alternative modeling strategies.

## 5. Explainability with SHAP

We use SHAP to explain the predictions of the best-performing model. This allows us to identify which features contribute most to expenditure prediction, 
both globally (feature importance) and locally (individual tourist profiles).

We could say that SHAP lets us peek inside our best model and understand how it makes predictions:

- **Globally:** see which features are most important overall  
- **Locally:** see how each feature affects individual predictions  

In [ ]:
sample_X = X.sample(50000, random_state=42)

In [ ]:
# Extract the trained regressor from the pipeline
reg = best_model.named_steps["regressor"]
preproc = best_model.named_steps["preprocessor"]

# Transform the sampled data
X_transformed = preproc.transform(sample_X)

# Use TreeExplainer (fast)
explainer = shap.TreeExplainer(reg)
shap_values = explainer(X_transformed)

The summary plots give a clear picture of what drives spending, making the model easier to explain to stakeholders or plug into a chatbot.

In [ ]:
# Summary plot
shap.summary_plot(shap_values, X_transformed, feature_names=preproc.get_feature_names_out())

## 6. Reporting Insights

With the models trained and saved, we can now:
- Assign new tourists to a cluster.
- Predict daily expenditure.
- Explain predictions with SHAP values.  

These elements will feed into the **MCP report server**, 
where the chatbot can generate dynamic reports for end-users.
